There are parts of the sample metadata in the CRC collections that should probably be related to other information sources. Values for geologic age and formation at depth should be associated with other subsurface geology data. Other sources of data may be tied to API well numbers. Operator names, well names, and field names may all be related to other sources as well. A look through current values in the CRC records shows evidence that they have been recorded often as a matter of course but not ever used for this kind of purpose (e.g., many alternate spellings of what is likely the same entity).

This notebook uses the final assempled cores and cuttings collections stashed in MongoDB to work out the basic "dimensions" of the data for the concepts that might be most useful in linking to other related information but are in need of come cleanup.

In [1]:
from pymongo import MongoClient
import pandas as pd
import json

mongo_ndc = MongoClient()

We can get a fairly quick look at the unique values for each property of interest by leveraging groupby in Pandas. The following codeblock works through the main string value properties from both the CRC original data and the surficial geology reference we brought in from the geologic mapping unit amalgamation provided by Macrostrat. Field name, unique value, and number of occurrences go into a dataframe that I drop out to a CSV for further reference.

In [2]:
root_fields = {
    "_id": 0,
    "surface_age" : 1, 
    "gmu_name" : 1, 
    "strat_unit" : 1, 
    "Operator" : 1, 
    "Well Name" : 1, 
    "Field" : 1 
}

all_data = [i for i in mongo_ndc.crc.cores.find({},root_fields)]
all_data.extend([i for i in mongo_ndc.crc.cuttings.find({},root_fields)])

df_all_data = pd.DataFrame(all_data)

count_stats = list()
for field in [k for k in root_fields.keys() if k != "_id"]:
    count_stats.extend({"stat_field": field, "unique_value": x, "count": len(y)} for x, y in df_all_data.groupby(field))

pipeline = [
    {
        u"$unwind": {
            u"path": u"$intervals"
        }
    }, 
    {
        u"$project": {
            u"formation_from_crc": u"$intervals.Formation",
            u"age_from_crc": u"$intervals.Age",
            u"_id": 0
        }
    }
]

crc_age_formation = [i for i in mongo_ndc.crc.cores.aggregate(pipeline)]
crc_age_formation.extend([i for i in mongo_ndc.crc.cuttings.aggregate(pipeline)])

df_crc_age_formation = pd.DataFrame(crc_age_formation)
df_crc_age_formation = df_crc_age_formation.dropna(axis=0, how='any')

count_stats.extend({"stat_field": "formation_from_crc", "unique_value": x, "count": len(y)} for x, y in df_crc_age_formation.groupby("formation_from_crc"))
count_stats.extend({"stat_field": "age_from_crc", "unique_value": x, "count": len(y)} for x, y in df_crc_age_formation.groupby("age_from_crc"))

pipeline = [
    {
        u"$project": {
            u"surface_rocktype": 1.0
        }
    }, 
    {
        u"$unwind": {
            u"path": u"$surface_rocktype"
        }
    }
]

gmu_rocktype = [i for i in mongo_ndc.crc.cores.aggregate(pipeline)]
gmu_rocktype.extend([i for i in mongo_ndc.crc.cuttings.aggregate(pipeline)])

df_gmu_rocktype = pd.DataFrame(gmu_rocktype)

count_stats.extend({"stat_field": "surface_rocktype", "unique_value": x, "count": len(y)} for x, y in df_gmu_rocktype.groupby("surface_rocktype"))

df_count_stats = pd.DataFrame(count_stats)
df_count_stats.to_csv('crcwc_facets.csv')

In [3]:
df_count_stats.head()

,stat_field,unique_value,count
0,surface_age,,85
1,surface_age,Albian,37
2,surface_age,Albian - Cenomanian,23
3,surface_age,Anisian - Carnian,17
4,surface_age,Aptian - Albian,2


Here are the fields we pulled together for further examination, output to a crcwc_facets.csv file. Each of these comes with their own challenges in terms of the actual meaning/significance of the values to the wells in the CRC Well Catalog that will require some further thought.

In [4]:
for field_name, group in df_count_stats.groupby("stat_field"):
    print(field_name)

Field
Operator
Well Name
age_from_crc
formation_from_crc
gmu_name
strat_unit
surface_age
surface_rocktype


If we want to examine a particular property from the facets, we can slice the dataframe like this.

In [5]:
df_count_stats.loc[df_count_stats.stat_field == "surface_rocktype"]

,stat_field,unique_value,count
68623,surface_rocktype,alluvium,2111
68624,surface_rocktype,amphibolite,53
68625,surface_rocktype,andesite,392
68626,surface_rocktype,anhydrite,18
68627,surface_rocktype,aplite,2
68628,surface_rocktype,arenite,92
68629,surface_rocktype,argillite,55
68630,surface_rocktype,arkose,582
68631,surface_rocktype,ash,352
68632,surface_rocktype,basalt,43


One interesting area to examine that has a little bit of traction in terms of standardized vocabularies is lithology or rock type. Theoretically, we should be able to find all of the rock type values pulled in from the Macrostrat API against geologic mapping information in the GeoSciML lithology vocabulary. I initially took a crack at trying to work with the search services available through the [CGI GeoScience Vocabularies](http://resource.geosciml.org/classifier/cgi/lithology) but ran into some technical and performance problems, so I cached a derivation of the identifiers, labels, and definitions in a local file [lithology_vocab.json](lithology_vocab.json) just to demonstrate a point.

Ultimately, I think we want individual collection owners and the NDC as a whole to be able to consult a whole set of managed ontologies on a regular basis to find alignment with concepts in source data. Alignment can be some type of direct connection between terms or a specified relationship to a concept in a vocabulary, potentially through an element of uncertainty if a search algorithim is used. As those relationships are built and expressed through the data, they form the basis for integration across disparate collections and new capabilities in search and discovery.

As a simple example of this, I run through a process below to pull up the snapshot of the lithology vocabulary, check for alignment with the rock type terms in our CRC records, add the term URI to those that match, and show the results (53 matching, 42 not). To make this real, a more sophisticated search needs to be developed, hopefully against an established and programmable source for the vocabularies. Further discussion on the utility of this kind of vocabulary alignment work should help to prioritize efforts.

In [6]:
with open("lithology_vocab.json", "r") as f:
    lithology_vocab = json.load(f)
    f.close()

In [7]:
lithology_vocab[:5]

[{'uri': 'http://resource.geosciml.org/classifier/cgi/lithology/acidic_igneous_material',
  'definition': 'Igneous material with more than 63 percent SiO2.',
  'label': 'acidic igneous material'},
 {'uri': 'http://resource.geosciml.org/classifier/cgi/lithology/acidic_igneous_rock',
  'definition': 'Igneous rock with more than 63 percent SiO2.',
  'label': 'acidic igneous rock'},
 {'uri': 'http://resource.geosciml.org/classifier/cgi/lithology/alkali-olivine_basalt',
  'definition': 'Alkali olivine basalt is silica-undersaturated, characterized by the absence of orthopyroxene, absence of quartz, presence of olivine, and typically contains some feldspathoid mineral, alkali feldspar or phlogopite in the groundmass. Feldspar phenocrysts typically are labradorite to andesine in composition. Augite is rich in titanium compared to augite in tholeiitic basalt. Alkali olivine basalt is relatively rich in sodium.',
  'label': 'alkali olivine basalt'},
 {'uri': 'http://resource.geosciml.org/classi

In [8]:
def lookup_rock_type(label):
    return next((i["uri"] for i in lithology_vocab if i["label"] == label), None)

In [9]:
df_count_stats["term_uri"] = df_count_stats.apply(lambda x: lookup_rock_type(x["unique_value"]), axis=1)

In [10]:
df_count_stats.loc[(df_count_stats.stat_field == "surface_rocktype") & (df_count_stats.term_uri.notnull())]

,stat_field,unique_value,count,term_uri
68624,surface_rocktype,amphibolite,53,http://resource.geosciml.org/classifier/cgi/li...
68625,surface_rocktype,andesite,392,http://resource.geosciml.org/classifier/cgi/li...
68627,surface_rocktype,aplite,2,http://resource.geosciml.org/classifier/cgi/li...
68628,surface_rocktype,arenite,92,http://resource.geosciml.org/classifier/cgi/li...
68632,surface_rocktype,basalt,43,http://resource.geosciml.org/classifier/cgi/li...
68633,surface_rocktype,breccia,39,http://resource.geosciml.org/classifier/cgi/li...
68635,surface_rocktype,chalk,83,http://resource.geosciml.org/classifier/cgi/li...
68637,surface_rocktype,clay,4229,http://resource.geosciml.org/classifier/cgi/li...
68638,surface_rocktype,claystone,11592,http://resource.geosciml.org/classifier/cgi/li...
68639,surface_rocktype,coal,8047,http://resource.geosciml.org/classifier/cgi/li...


In [11]:
df_count_stats.loc[(df_count_stats.stat_field == "surface_rocktype") & (df_count_stats.term_uri.isnull())]

,stat_field,unique_value,count,term_uri
68623,surface_rocktype,alluvium,2111,None
68626,surface_rocktype,anhydrite,18,None
68629,surface_rocktype,argillite,55,None
68630,surface_rocktype,arkose,582,None
68631,surface_rocktype,ash,352,None
68634,surface_rocktype,carbonate,982,None
68636,surface_rocktype,chert,1082,None
68640,surface_rocktype,colluvium,334,None
68642,surface_rocktype,coquina,155,None
68644,surface_rocktype,diabase,3,None


One other potential point of interest from the CRC catalog that represents an untapped resource is the collection of "analysis" files that are linked to cores and cuttings. As part of our data amalgamation, we brought all of these together and can examine what we may have. The following codeblock simply pulls all the document links together and splits out their file extensions so we can pull a quick summary. The collection of documents is really a hodgepodge of various types of things, some of which do represent analytical results from various types of analyses on the samples, but others are simply related artifacts that have been collected and stored over time by the CRC.

In [12]:
pipeline = [
    {
        u"$project": {
            u"documents": 1.0
        }
    }, 
    {
        u"$unwind": {
            u"path": u"$documents"
        }
    }
]

all_documents = [i for i in mongo_ndc.crc.cores.aggregate(pipeline)]
all_documents.extend([i for i in mongo_ndc.crc.cuttings.aggregate(pipeline)])
df_documents = pd.DataFrame(all_documents)
df_documents["filename"] = df_documents.apply(lambda x: x.documents.split("/")[-1], axis=1)
df_documents["extension"] = df_documents.apply(lambda x: x.documents.split(".")[-1].lower(), axis=1)

for extension, group in df_documents.groupby("extension"):
    print(extension, len(group))

 3
bmd 154
bmp 1
csv 106
db 3
doc 57
docx 119
emf 28
hdr 41
jpg 590
las 6
mp4 3
pdf 8741
png 34
ppt 65
pptx 79
psd 168
tif 2141
tiff 1
txt 5
xls 2115
xlsb 3
xlsm 178
xlsx 3406
xps 2
zip 72


Some of these files represent more potentially exploitable material than others. The following codeblocks dump out just the CSV results and then pull up a couple of random examples via Pandas. We could run through a process to try and read as many files as possible from this system, summarizing their contents and adding further metadata to our catalog. Pulling all of what we could extract in this fashion would at least generate some type of master registry of analytes and other properties that could be further examined through semantic alignment work. 

In [13]:
df_documents.loc[df_documents.extension == "csv"]

,_id,documents,filename,extension
91,5d87a63b0863b1fac6a7cba1,http://my.usgs.gov/crc-data/core/16106/core_an...,core_analysis_88604_T038data09.csv,csv
468,5d87a63b0863b1fac6a7cd1c,http://my.usgs.gov/crc-data/core/77935/core_an...,core_analysis_108350_S505XRFData01.csv,csv
732,5d87a63b0863b1fac6a7cda1,http://my.usgs.gov/crc-data/prod/core/99399/co...,core_analysis_122480_S452BioStratData01.csv,csv
752,5d87a63b0863b1fac6a7cdbc,http://my.usgs.gov/crc-data/core/78910/core_an...,core_analysis_80308_T481Data01.csv,csv
1295,5d87a63b0863b1fac6a7cf77,http://my.usgs.gov/crc-data/core/9209/core_ana...,core_analysis_88593_B136data08.csv,csv
1335,5d87a63b0863b1fac6a7cf8d,http://my.usgs.gov/crc-data/core/14652/core_an...,core_analysis_83822_W074Geochem01.csv,csv
1554,5d87a63b0863b1fac6a7d044,http://my.usgs.gov/crc-data/core/13657/core_an...,core_analysis_80164_S601data7.csv,csv
1571,5d87a63b0863b1fac6a7d04e,http://my.usgs.gov/crc-data/core/11925/core_an...,core_analysis_88601_E190data05.csv,csv
2463,5d87a63b0863b1fac6a7d2bd,http://my.usgs.gov/crc-data/prod/core/17558/co...,core_analysis_123431_V248GeoChemData01.csv,csv
2793,5d87a63b0863b1fac6a7d3dd,http://my.usgs.gov/crc-data/core/16121/core_an...,core_analysis_113056_T055-Data02.csv,csv


This example, picked at random, indicates a couple of problems often encountered with reading CSV data - dealing with different text encoding from CSV generated from Windows operating systems, in particular, and finding the actual data table within what may be a mixed bag of header information. There are various messy data tools that help to deal with some of these problems. The Excel files are sometimes even more challenging to deal with as they often contain a whole lot of additional metadata about the data file structured in various ways that can be difficult to handle without very specific software.

In [14]:
pd.read_csv(df_documents.iloc[12264]["documents"], encoding='latin1')

,Sample data for well: 1 McTuillin Federal B703,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135
0,depth,type,analyst,label,Bisaccate pollen,Chatangiella spp.,Circulodinium spp.,Circulodinium distinctum,Cribroperidinium spp.,Exochosphaeridium spp.,...,Laevigatosporites gracilis,Pterospermella zonaria,Dinopterygium konradense,Florentinia cooksoniae,"Genus A sp. 1 Upshaw, 1964",Klukisporites spp.,Isabelidinium? amphiatum,Rouseisporites reticulatus,Scriniodinium `simplicum` ROB,?Florentinia mantellii
1,NaN,NaN,NaN,NaN,MI,DC,DC,DC,DC,DC,...,MI,AL,DC,DC,MI,MI,DC,MI,DC,DC
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4824.7,CO,JPGF,NaN,32,2,1,3,+,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4827,CO,JPGF,NaN,61,4,NaN,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4828.3,CO,JPGF,NaN,38,3,NaN,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4838.2,CO,JPGF,NaN,61,5,NaN,1,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4844.7,CO,JPGF,NaN,54,5,2,4,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4849,CO,JPGF,NaN,82,2,NaN,+,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4851.3,CO,JPGF,NaN,92,2,1,3,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This example is reasonably typical of geochemical analyses where we have some identifiers that are likely not referenced in the actual CRC metadata, depth values where subsamples were taken, and analyte names that need to be deciphered.

In [15]:
pd.read_csv(df_documents.iloc[3537]["documents"])

,Sample,Sample_id1,Sample_id2,Depth_ft,C_pct,H_pct,N_pct,H/C,N/C,tot_Kerogen_S_pct,tot_Kerogen_S/C,Pyritic_Fe_pct,Pyritic_S_pct,Organic_S_pct,Organic_S/C
0,E150503-001,HP7-149.5,7 HP Beaver,149.5,57.45,5.26,1.93,1.09,0.03,5.54,0.04,2.25,2.58,2.96,0.02
1,E150503-002,HP2-242.9,2 HP Potter,242.9,53.58,4.92,1.60,1.09,0.03,5.22,0.04,1.76,2.02,3.20,0.02
2,E150503-003,BN-4536,1-3 BN Coastal,4536.0,64.53,5.97,2.16,1.10,0.03,4.44,0.03,2.28,2.62,1.82,0.01
3,E150503-004,HP3-272.5,3 HP Heath,272.5,56.64,5.23,1.88,1.10,0.03,6.90,0.05,3.02,3.47,3.43,0.02
